# Introduction to running PROCESS

A Jupyter notebook to demonstrate usage of the `process` package.

## Motivation
Process is moving away from being a runnable package with a single command-line entry-point to an importable package which can be scripted. This notebook is a good way of demonstrating the functionality of the package, and could provide a better way of working for modellers, who may wish to create their own scripts or notebooks for different tasks.

## Setup
Currently the various classes and "utilities" scripts in Process have different interfaces and read and write files in differing manners and in locations that can't be easily controlled. To partially avoid the headaches associated with this, the code cell below defines a function to allow each example to be run in a temporary directory, much like a test. Input files are copied to this temporary directory and outputs contained there before the directory is removed.

This temporary directory function is only required for running the examples below and removing any modifications afterwards, not in regular use of Process where the outputs will want to be preserved. Further development work will unify these disparate ways of running Process into a common Pythonic form.

In [ ]:
%load_ext autoreload
%autoreload 2
%pip install pillow

from pathlib import Path
from shutil import copy
from tempfile import TemporaryDirectory

# Define project root dir; when running a notebook, the cwd is the dir the notebook is in
PROJ_DIR = Path.cwd().parent


def copy_to_temp_dir(input_rel):
    """Copy an input file to a new temp dir and return its new path.

    The new TemporaryDirectory object is returned to avoid destruction of the
    object, which results in deletion of the directory prematurely. This way
    the cleanup() method can be used to delete the directory when required.
    :param input_rel: file path relative to project root dir
    :type input_rel: str
    :return: temporary dir and absolute path to file in temp dir
    :rtype: (TemporaryDirectory, pathlib.Path)
    """
    # Create temporary dir to contain the run's outputs
    temp_dir = TemporaryDirectory()
    temp_dir_path = Path(temp_dir.name)

    # Define absolute path for input file
    input_rel_path = Path(input_rel)
    input_abs_path = PROJ_DIR / input_rel_path

    try:
        assert input_abs_path.exists()
    except AssertionError as err:
        raise FileNotFoundError("Input file doesn't exist.") from err

    # Copy input file to temp dir
    copy(input_abs_path, temp_dir_path)
    temp_input_path = temp_dir_path / input_abs_path.name

    return temp_dir, temp_input_path, temp_dir_path


## Basic run of Process
Run Process on an input file using the `SingleRun` class. This outputs an `MFILE.DAT` and an `OUT.DAT`.

In [ ]:
from process.main import SingleRun, setup_loggers

# setup the loggers so that the output is not spammed with model errors/warnings
setup_loggers()

# Define input file name relative to project dir, then copy to temp dir
script_dir = Path("__file__").parent.resolve()
input_rel = script_dir / "data/large_tokamak_IN.DAT"

temp_dir, temp_input_path, temp_dir_path = copy_to_temp_dir(input_rel)

# Run process on an input file in a temporary directory
single_run = SingleRun(str(temp_input_path))
single_run.run()

## Plot summary
Create a summary of the generated `MFILE.DAT` using `plot_proc`.

In [ ]:
from process.io import plot_proc

# plot_proc uses command line arguments of the current process. Jupyter adds command line arguments under the hood causing plot_proc to fail. running plot proc in its own process isolates it from the jupyter command line arguments
# Use the --output_format argument "png" to create png outputs. pdf output is also available and is the default
plot_proc.main(args=["-f", str(single_run.mfile_path), "--output_format", "png"])

`plot_proc`'s output.

In [ ]:
from IPython.display import display  # noqa: A004
from PIL import Image

img1 = Image.open(temp_dir_path / "large_tokamak_MFILE_SUMMARY/page0.png")
display(img1)
img2 = Image.open(temp_dir_path / "large_tokamak_MFILE_SUMMARY/page1.png")
display(img2)
img3 = Image.open(temp_dir_path / "large_tokamak_MFILE_SUMMARY/page2.png")
display(img3)

In [ ]:
# Delete temp dir
temp_dir.cleanup()

## View key output variables
Run the large tokamak scenario using `SingleRun` to set some values on the `CostModel` instance and then print them.

In [ ]:
# Define input file name relative to project dir
input_rel = script_dir / "data/large_tokamak_IN.DAT"
print(input_rel)
temp_dir, temp_input_path, _ = copy_to_temp_dir(input_rel)

# Run process on an input file
single_run = SingleRun(str(temp_input_path))
single_run.run()

In [ ]:
import process.data_structure

# Print some values on the CostModel instance
print(f"Heat transport system: {process.data_structure.cost_variables.c226:.3e} M$")
print(f"Electrical plant equipment: {process.data_structure.cost_variables.c24:.3e} M$")

In [ ]:
# Clean up
temp_dir.cleanup()

## VaryRun
Vary iteration parameters until a feasible solution is found, using the `VaryRun` class.

In [ ]:
import os

from process.main import VaryRun

input_rel = script_dir / "data/run_process.conf"
temp_dir, temp_input_path, _ = copy_to_temp_dir(input_rel)

# .conf file relies on a separate input file too; copy this as well
# TODO This double input file requirement needs to be removed
input_rel_2 = script_dir / "data/large_tokamak_IN.DAT"
copy(PROJ_DIR / input_rel_2, temp_dir.name)

# VaryRun uses process_config.py, which changes the current working directory
# via os.chdir() to the temporary dir. Apart from being bad practice, once the
# temp dir is removed, this causes Path.cwd() (as used in plot_scans.py) to
# throw an exception when trying to return the (now deleted) CWD. Hence it
# needs to be set back after VaryRun()
# TODO Remove the os.chdir() from VaryRun
cwd = Path.cwd()

vary_run = VaryRun(str(temp_input_path))
vary_run.run()
os.chdir(cwd)

temp_dir.cleanup()